In [1]:
duration <- function(fluxo, taxa, periodo = length(fluxo)){
  periodo <- 1:periodo
  duratio <- (sum((fluxo/(1+taxa)^periodo)*periodo)/sum((fluxo/(1+taxa)^periodo)))
  return(duratio)
}
duration_mod <- function(duration, nova_taxa){
    duratio_mod <- -duration/(1+nova_taxa)
    return(duratio_mod)
}

In [2]:
vpl <- function(fluxo, taxa, investimento, periodo = length(fluxo)){
  periodo <- c(1:periodo)
  vpl <- sum(fluxo/((1+taxa)^periodo))-investimento
  return(vpl)
}
tir_mc <- function(fluxo, n_taxas = 10000, investimento, periodo = length(fluxo),
                taxa_min = -0.99999, taxa_max = 1){
  taxas <- seq(taxa_min,taxa_max,l = n_taxas)
  tir1 <- sapply(taxas,vpl, investimento = investimento, 
                 fluxo = fluxo, periodo = length(fluxo))
  taxas[which.min(abs(tir1))]
}
tir <- function(fluxo, investimento){
  vpl <- function(fluxo, taxa, investimento, periodo = length(fluxo)){
    periodo <- c(1:periodo)
    vpl <- sum(fluxo/((1+taxa)^periodo))-investimento
    return(vpl)
  }
  tir <- Re(1/polyroot(c(-investimento, fluxo))-1)
  tir1 <- numeric(length(tir))
  for(i in 1:length(tir1)){
    tir1[i] <- round(vpl(fluxo, tir[i], investimento),7)
  }
  tir <- tir[which(abs(tir1) == min(abs(tir1)))]
  tir <- tir[which.max(tir)]
  erro <- vpl(fluxo,tir,investimento)
  lista <- list(Rate = tir, Error = erro)
  return(lista)
}

In [3]:
#Coeficiente de prestação
coef_pre <- function(r,n,x,p_est = NULL){
  if(is.null(p_est)){
    r/(1-(1+r)^(-n))*x
  }else{
    r/(1-(1+r)^(-n))*x-p_est
  }
}
r_presta <- function(n,x,p_est){
  f <- c(rep(p_est,n),-x)
  t <- Re(polyroot(f))-1
  coefi <- round(coef_pre(t,n,x)-p_est,6)
  mini <- which(coefi == min(abs(coefi)))
  t <- t[mini]
  t <- t[which.max(t)]
  coe <- coef_pre(t,n,x)
  lista <- list(Rate = t, Error = abs(coe)/p_est-1)
  lista
}

In [4]:
fluxo <- c(rep(0.1,10),1.1)
periodo <- 11
taxa <- 0.1
dur <- duration(fluxo = fluxo, periodo = periodo, taxa = taxa)
dur

[1] 7.144567

In [5]:
dur_mod <- duration_mod(duration = dur, 0.08)
dur_mod

[1] -6.61534

In [7]:
investimento <- 100
fluxo <- 100*cumprod(rnorm(20, mean = 0.9, sd = 0.1))
taxa <- tir(fluxo, investimento)
taxa
round(vpl(fluxo = fluxo,taxa = taxa$Rate, investimento = investimento),10)

$Rate
[1] 0.7433796

$Error
[1] 1.421085e-14

[1] 0

In [14]:
taxa_m <- tir_mc(fluxo = fluxo, n_taxas = 100000, investimento = investimento)
taxa_m

[1] 0.743379

In [16]:
vpl(fluxo = fluxo,taxa = taxa_m,investimento = investimento)

[1] 6.631997e-05